## Params

In [1]:
seed = 0

class args:
    num_layers = 5
    device = 'cuda:0'
    log_steps = 1
    hidden_channels = 64*4
    dropout = 0.2
    lr = 0.01
    epochs = 1000
    eval_steps = 10
    runs = 1

## Install prerequisites

In [2]:
import time
import humanize
start_time = time.time()

In [3]:
# install Open Graph Benchmark
! pip install ogb

# install PyTorch Geometric
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

     |████████████████████████████████| 45 kB 1.2 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 12.3 MB 24.9 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 21.6 MB 12.8 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 21.3 MB 12.8 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 6.3 MB 13.3 MB/s 
You should consider u

## Import libraries

In [4]:
import torch
from torch.nn import Parameter
import torch.nn.functional as F

from torch.optim.lr_scheduler import MultiStepLR,CosineAnnealingLR

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

## Logger

In [5]:
class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [6]:
device = f'{args.device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = PygNodePropPredDataset(name='ogbn-proteins',
                                 transform=T.ToSparseTensor())
data = dataset[0]

# Move edge features to node features.
data.x = data.adj_t.mean(dim=1)
data.adj_t.set_value_(None)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)
data = data.to(device)

Downloaded 0.21 GB: 100%|██████████| 216/216 [00:14<00:00, 14.55it/s]


Extracting dataset/proteinfunc.zip
Processing...
Loading necessary files...
This might take a while.


  0%|          | 0/1 [00:00<?, ?it/s]

Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 384.69it/s]


Converting graphs into PyG objects...
Saving...
Done!


In [7]:
print(humanize.naturaldelta(start_time - time.time()))

4 minutes


## Model Definition

In [8]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(
            GCNConv(in_channels, hidden_channels, normalize=False))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, normalize=False))
        self.convs.append(
            GCNConv(hidden_channels*2, out_channels, normalize=False))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        x = self.convs[0](x, adj_t)
        x_ = x
        for conv in self.convs[1:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = torch.cat([x_,x],1)    
        x = self.convs[-1](x, adj_t)
        return x

## Training and eval

In [9]:
def train(model, data, train_idx, optimizer):
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = criterion(out, data.y[train_idx].to(torch.float))
    loss.backward()
    optimizer.step()

    return loss.item()

In [10]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    y_pred = model(data.x, data.adj_t)

    train_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['rocauc']
    valid_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['rocauc']
    test_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc

In [11]:
# Pre-compute GCN normalization.
adj_t = data.adj_t.set_diag()
deg = adj_t.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj_t = deg_inv_sqrt.view(-1, 1) * adj_t * deg_inv_sqrt.view(1, -1)
data.adj_t = adj_t
    
evaluator = Evaluator(name='ogbn-proteins')
logger = Logger(args.runs, args)
best_test_score = 0


for seed in range(10):
    print("Seed = ",seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    model = GCN(data.num_features, args.hidden_channels, 112, args.num_layers, args.dropout).to(device)
    model.reset_parameters()
    for run in range(args.runs):        
        optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):

            loss = train(model, data, train_idx, optimizer)

            if epoch % args.eval_steps == 0:
                result = test(model, data, split_idx, evaluator)
                logger.add_result(run, result)

                if epoch % args.log_steps == 0:                
                    train_rocauc, valid_rocauc, test_rocauc = result
                    print(f'Run: {run + 1:02d}\t '
                          f'Epoch: {epoch:02d}\t '
                          f'Loss: {loss:.4f}\t '
                          f'Train: {100 * train_rocauc:.2f}%\t '
                          f'Valid: {100 * valid_rocauc:.2f}%\t'
                          f'Test: {100 * test_rocauc:.2f}%')
                    if(test_rocauc > best_test_score):
                        best_test_score = test_rocauc
                        save_path = "gcn.pth"
                        torch.save(model, save_path)
                        print("Model saved.")
        # logger.print_statistics(run)
        torch.cuda.empty_cache()

    # logger.print_statistics()

Seed =  0
Run: 01	 Epoch: 10	 Loss: 0.5319	 Train: 37.78%	 Valid: 35.44%	Test: 37.91%
Model saved.
Run: 01	 Epoch: 20	 Loss: 0.4211	 Train: 36.71%	 Valid: 33.45%	Test: 35.85%
Run: 01	 Epoch: 30	 Loss: 0.4018	 Train: 39.05%	 Valid: 33.44%	Test: 34.93%
Run: 01	 Epoch: 40	 Loss: 0.3900	 Train: 43.59%	 Valid: 34.99%	Test: 35.17%
Run: 01	 Epoch: 50	 Loss: 0.3792	 Train: 48.62%	 Valid: 36.14%	Test: 35.75%
Run: 01	 Epoch: 60	 Loss: 0.3733	 Train: 52.07%	 Valid: 38.32%	Test: 37.14%
Run: 01	 Epoch: 70	 Loss: 0.3670	 Train: 54.18%	 Valid: 40.72%	Test: 38.41%
Model saved.
Run: 01	 Epoch: 80	 Loss: 0.3628	 Train: 56.26%	 Valid: 44.23%	Test: 40.50%
Model saved.
Run: 01	 Epoch: 90	 Loss: 0.3552	 Train: 58.44%	 Valid: 48.41%	Test: 42.91%
Model saved.
Run: 01	 Epoch: 100	 Loss: 0.3517	 Train: 59.78%	 Valid: 51.96%	Test: 43.58%
Model saved.
Run: 01	 Epoch: 110	 Loss: 0.3484	 Train: 61.41%	 Valid: 52.65%	Test: 45.82%
Model saved.
Run: 01	 Epoch: 120	 Loss: 0.3418	 Train: 62.88%	 Valid: 55.85%	Test: 46.9

In [12]:
print(best_test_score*100)

75.38901496723601


In [13]:
print(humanize.naturaldelta(start_time - time.time()))

3 hours


In [14]:
# del model, optimizer
# torch.cuda.empty_cache()